In [1]:
import pandas as pd
import sys


In [2]:
sys.path.append('../../../../')  # Adiciona o diretório raiz ao path para importar conn.py
from conn import get_connection
engine = get_connection()


##### PDI

In [11]:
query_pdi = """
    SELECT * FROM byd.db_compras_byd """
df_pdi = pd.read_sql_query(query_pdi, engine)

C:\Users\giovanni.5683\AppData\Local\Temp\ipykernel_22256\982354752.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pdi = pd.read_sql_query(query_pdi, engine)


DatabaseError: Execution failed on sql: 
    SELECT * FROM byd.db_compras_byd 
server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.

unable to rollback

In [ ]:
df_pdi

,origem,nome_empresa,chv_empresa,bandeira_bi,ordem_bandeira,sistema,revenda,idnfexterno,numero_nota_fiscal,serie_nota_fiscal,...,dta_recebimento_atacado,dta_recebimento_performance,detalhes,status_pdi,status_atacado,status_performance,ref_comunicado_pdi,ref_comunicado_atacado,ref_comunicado_performance,modelo_format
0,APOLLO,25.6 BYD CASCAVEL,A25_6,BYD,D,A,6,A25657365V010,5736,5,...,None,2025-12-08,None,PENDENTE,PENDENTE,PENDENTE,None,None,None,BYD SONG PRO
1,APOLLO,25.6 BYD CASCAVEL,A25_6,BYD,D,A,6,A25657375V010,5737,5,...,None,2025-12-08,None,PENDENTE,PENDENTE,PENDENTE,None,None,None,BYD SONG PRO
2,APOLLO,25.6 BYD CASCAVEL,A25_6,BYD,D,A,6,A25657385V010,5738,5,...,None,2025-12-08,None,PENDENTE,PENDENTE,PENDENTE,None,None,None,BYD SONG PRO
3,APOLLO,25.6 BYD CASCAVEL,A25_6,BYD,D,A,6,A25657395V010,5739,5,...,None,2025-12-08,None,PENDENTE,PENDENTE,PENDENTE,None,None,None,BYD SONG PRO
4,APOLLO,25.6 BYD CASCAVEL,A25_6,BYD,D,A,6,A25659595V010,5959,5,...,None,None,None,PENDENTE,PENDENTE,PENDENTE,None,None,None,BYD DOLPHIN MINI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4222,APOLLO,25.9 BYD AUTOPLUS UMUARAMA,A25_9,BYD,D,A,9,A25991805V010,9180,5,...,2025-05-28 00:00:00,None,None,PENDENTE,PENDENTE,PENDENTE,None,None,None,BYD KING GS
4223,APOLLO,25.9 BYD AUTOPLUS UMUARAMA,A25_9,BYD,D,A,9,A25991795V010,9179,5,...,2025-05-28 00:00:00,None,None,PENDENTE,PENDENTE,PENDENTE,None,None,None,BYD KING GS
4224,APOLLO,25.9 BYD AUTOPLUS UMUARAMA,A25_9,BYD,D,A,9,A25991785V010,9178,5,...,2025-05-28 00:00:00,None,None,PENDENTE,PENDENTE,PENDENTE,None,None,None,BYD KING GS
4225,APOLLO,25.9 BYD AUTOPLUS UMUARAMA,A25_9,BYD,D,A,9,A25991775V010,9177,5,...,2025-05-28 00:00:00,None,None,PENDENTE,PENDENTE,PENDENTE,None,None,None,BYD KING GS


In [ ]:
# Carregar o arquivo CSV
df_csv = pd.read_csv('../../../../Datasets/pdi.csv')

# Verificar quais chassis do CSV estão no df_pdi
chassis_csv = df_csv['chassi'].dropna().unique()
chassis_pdi = df_pdi['chassi'].dropna().unique()

# Encontrar os chassis que estão em ambos
chassis_comuns = set(chassis_csv) & set(chassis_pdi)

print(f"Total de chassis no CSV: {len(chassis_csv)}")
print(f"Total de chassis no df_pdi: {len(chassis_pdi)}")
print(f"Chassis que estão em ambos: {len(chassis_comuns)}")
print(f"\nPercentual de chassis do CSV presentes no df_pdi: {len(chassis_comuns) / len(chassis_csv) * 100:.2f}%")

Total de chassis no CSV: 469
Total de chassis no df_pdi: 4201
Chassis que estão em ambos: 469

Percentual de chassis do CSV presentes no df_pdi: 100.00%


In [ ]:
# Encontrar chassi em comum
chassis_comuns = set(df_csv['chassi'].dropna()) & set(df_pdi['chassi'].dropna())
df_csv_comuns = df_csv[df_csv['chassi'].isin(chassis_comuns)].copy()

# Ajuste de tipos conforme necessário
df_csv_comuns['val_pdi'] = df_csv_comuns['val_pdi'].str.replace(',', '.').astype(float)
df_csv_comuns['dta_recebimento_pdi'] = pd.to_datetime(df_csv_comuns['dta_recebimento_pdi'], format='%d/%m/%Y')

# Defina as colunas que existem tanto no CSV quanto no banco
colunas_banco = set(df_pdi.columns)
colunas_csv = set(df_csv_comuns.columns)
colunas_comuns = list(colunas_banco & colunas_csv)
colunas_comuns.remove('chassi')  # 'chassi' será usado no WHERE

# Atualizar cada registro usando psycopg2
cursor = engine.cursor()
try:
    for _, row in df_csv_comuns.iterrows():
        set_clause = ', '.join([f"{col} = %s" for col in colunas_comuns])
        update_query = f"UPDATE byd.db_compras_byd SET {set_clause} WHERE chassi = %s"
        values = [row[col] for col in colunas_comuns] + [row['chassi']]
        cursor.execute(update_query, values)
    
    engine.commit()
    print(f"Atualização concluída! {len(df_csv_comuns)} registros atualizados.")
except Exception as e:
    engine.rollback()
    print(f"Erro durante atualização: {e}")
    raise
finally:
    cursor.close()

Atualização concluída! 469 registros atualizados.


##### ATACADO

In [13]:
query_atacado = """
    SELECT * FROM byd.db_compras_byd """
df_atacado = pd.read_sql_query(query_atacado, engine)

C:\Users\giovanni.5683\AppData\Local\Temp\ipykernel_22256\167893553.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_atacado = pd.read_sql_query(query_atacado, engine)


InterfaceError: connection already closed

In [ ]:
# Carregar o arquivo CSV
df_csv_atacado = pd.read_csv('../../../../Datasets/atacado.csv')

# Verificar quais chassis do CSV estão no df_pdi
chassis_csv = df_csv_atacado['chassi'].dropna().unique()
chassis_atacado = df_atacado['chassi'].dropna().unique()
# Encontrar os chassis que estão em ambos
chassis_comuns = set(chassis_csv) & set(chassis_atacado)

print(f"Total de chassis no CSV: {len(chassis_csv)}")
print(f"Total de chassis no df_atacado: {len(chassis_atacado)}")
print(f"Chassis que estão em ambos: {len(chassis_comuns)}")
print(f"\nPercentual de chassis do CSV presentes no df_atacado: {len(chassis_comuns) / len(chassis_csv) * 100:.2f}%")

Total de chassis no CSV: 549
Total de chassis no df_atacado: 4201
Chassis que estão em ambos: 548

Percentual de chassis do CSV presentes no df_atacado: 99.82%


In [ ]:
# Encontrar chassi em comum
chassis_comuns = set(df_csv_atacado['chassi'].dropna()) & set(df_atacado['chassi'].dropna())
df_csv_comuns = df_csv_atacado[df_csv_atacado['chassi'].isin(chassis_comuns)].copy()

# Ajuste de tipos conforme necessário
df_csv_comuns['pps'] = df_csv_comuns['pps'].str.replace('.', '').str.replace(',', '.').astype(float)
df_csv_comuns['percentil_atacado'] = df_csv_comuns['percentil_atacado'].str.replace(',', '.').astype(float)
df_csv_comuns['dta_recebimento_atacado'] = pd.to_datetime(df_csv_comuns['dta_recebimento_atacado'], format='%d/%m/%Y')

# Defina as colunas que existem tanto no CSV quanto no banco
colunas_banco = set(df_atacado.columns)
colunas_csv = set(df_csv_comuns.columns)
colunas_comuns = list(colunas_banco & colunas_csv)
colunas_comuns.remove('chassi')  # 'chassi' será usado no WHERE

# Atualizar cada registro usando psycopg2
cursor = engine.cursor()
try:
    for _, row in df_csv_comuns.iterrows():
        set_clause = ', '.join([f"{col} = %s" for col in colunas_comuns])
        update_query = f"UPDATE byd.db_compras_byd SET {set_clause} WHERE chassi = %s"
        values = [row[col] for col in colunas_comuns] + [row['chassi']]
        cursor.execute(update_query, values)
    
    engine.commit()
    print(f"Atualização concluída! {len(df_csv_comuns)} registros atualizados.")
except Exception as e:
    engine.rollback()
    print(f"Erro durante atualização: {e}")
    raise
finally:
    cursor.close()

Atualização concluída! 548 registros atualizados.


##### PERFORMANCE

In [29]:
query_performance = """
    SELECT * FROM byd.db_compras_byd """
df_performance = pd.read_sql_query(query_performance, engine)

C:\Users\giovanni.5683\AppData\Local\Temp\ipykernel_22500\272614940.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_performance = pd.read_sql_query(query_performance, engine)


In [5]:
# Carregar 
df_csv_performance = pd.read_csv('../../../../Datasets/performance.csv')

# Verificar quais chassis do CSV estão no df_pdi
chassis_csv = df_csv_performance['chassi'].dropna().unique()
chassis_performance = df_performance['chassi'].dropna().unique()
# Encontrar os chassis que estão em ambos
chassis_comuns = set(chassis_csv) & set(chassis_performance)

print(f"Total de chassis no CSV: {len(chassis_csv)}")
print(f"Total de chassis no df_performance: {len(chassis_performance)}")
print(f"Chassis que estão em ambos: {len(chassis_comuns)}")
print(f"\nPercentual de chassis do CSV presentes no df_performance: {len(chassis_comuns) / len(chassis_csv) * 100:.2f}%")

NameError: name 'df_performance' is not defined

In [33]:
# Encontrar chassi em comum
chassis_comuns = set(df_csv_performance['chassi'].dropna()) & set(df_performance['chassi'].dropna())
df_csv_comuns = df_csv_performance[df_csv_performance['chassi'].isin(chassis_comuns)].copy()

# Ajuste de tipos conforme necessário
# Converter pps (se existir e não for NaN)
if 'pps' in df_csv_comuns.columns:
    df_csv_comuns['pps'] = df_csv_comuns['pps'].str.replace('R$', '').str.strip().str.replace('.', '').str.replace(',', '.').astype(float)

# Converter percentis (removendo vírgula e convertendo para float)
percentil_cols = ['percentil_tiktok', 'percentil_nps', 'percentil_visits', 
                  'percentil_snapon', 'percentil_mystery', 'percentil_chatbot', 'percentil_download']

for col in percentil_cols:
    if col in df_csv_comuns.columns:
        df_csv_comuns[col] = df_csv_comuns[col].str.replace(',', '.').astype(float)

# Adicionar data de recebimento performance
df_csv_comuns['dta_recebimento_performance'] = pd.Timestamp.now().date()

# Defina as colunas que existem tanto no CSV quanto no banco
colunas_banco = set(df_performance.columns)
colunas_csv = set(df_csv_comuns.columns)
colunas_comuns = list(colunas_banco & colunas_csv)
colunas_comuns.remove('chassi')  # 'chassi' será usado no WHERE

# Atualizar cada registro usando psycopg2
cursor = engine.cursor()
try:
    for _, row in df_csv_comuns.iterrows():
        set_clause = ', '.join([f"{col} = %s" for col in colunas_comuns])
        update_query = f"UPDATE byd.db_compras_byd SET {set_clause} WHERE chassi = %s"
        values = [row[col] for col in colunas_comuns] + [row['chassi']]
        cursor.execute(update_query, values)
    
    engine.commit()
    print(f"Atualização concluída! {len(df_csv_comuns)} registros atualizados.")
except Exception as e:
    engine.rollback()
    print(f"Erro durante atualização: {e}")
    raise
finally:
    cursor.close()

Atualização concluída! 1448 registros atualizados.


In [32]:
conn.rollback()

##### OUTUBRO

In [34]:
query_outubro = """
    SELECT * FROM byd.db_compras_byd """
df_outubro = pd.read_sql_query(query_outubro, engine)

C:\Users\giovanni.5683\AppData\Local\Temp\ipykernel_22500\2505401186.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_outubro = pd.read_sql_query(query_outubro, engine)


In [36]:
# Carregar o arquivo CSV
df_csv_outubro = pd.read_csv('../../../../Datasets/outubro.csv')

# Verificar quais chassis do CSV estão no df_pdi
chassis_csv = df_csv_outubro['chassi'].dropna().unique()
chassis_performance = df_outubro['chassi'].dropna().unique()
# Encontrar os chassis que estão em ambos
chassis_comuns = set(chassis_csv) & set(chassis_performance)

print(f"Total de chassis no CSV: {len(chassis_csv)}")
print(f"Total de chassis no df_outubro: {len(chassis_performance)}")
print(f"Chassis que estão em ambos: {len(chassis_comuns)}")
print(f"\nPercentual de chassis do CSV presentes no df_outubro: {len(chassis_comuns) / len(chassis_csv) * 100:.2f}%")

Total de chassis no CSV: 733
Total de chassis no df_outubro: 4201
Chassis que estão em ambos: 649

Percentual de chassis do CSV presentes no df_outubro: 88.54%


In [37]:
# Encontrar chassi em comum
chassis_comuns = set(df_csv_outubro['chassi'].dropna()) & set(df_outubro['chassi'].dropna())
df_csv_comuns = df_csv_outubro[df_csv_outubro['chassi'].isin(chassis_comuns)].copy()

# Ajuste de tipos conforme necessário
# Converter pps (removendo pontos e vírgulas)
if 'pps' in df_csv_comuns.columns:
    df_csv_comuns['pps'] = df_csv_comuns['pps'].str.replace('.', '').str.replace(',', '.').astype(float)

# Converter val_pdi para float
if 'val_pdi' in df_csv_comuns.columns:
    df_csv_comuns['val_pdi'] = df_csv_comuns['val_pdi'].astype(float)

# Converter percentil_atacado
if 'percentil_atacado' in df_csv_comuns.columns:
    df_csv_comuns['percentil_atacado'] = df_csv_comuns['percentil_atacado'].str.replace(',', '.').astype(float)

# Adicionar data de recebimento de outubro
df_csv_comuns['dta_recebimento_outubro'] = pd.Timestamp.now().date()

# Defina as colunas que existem tanto no CSV quanto no banco
colunas_banco = set(df_outubro.columns)
colunas_csv = set(df_csv_comuns.columns)
colunas_comuns = list(colunas_banco & colunas_csv)

# Remover 'chassi' pois será usado no WHERE
if 'chassi' in colunas_comuns:
    colunas_comuns.remove('chassi')

# Atualizar cada registro usando psycopg2
cursor = engine.cursor()
try:
    for _, row in df_csv_comuns.iterrows():
        set_clause = ', '.join([f"{col} = %s" for col in colunas_comuns])
        update_query = f"UPDATE byd.db_compras_byd SET {set_clause} WHERE chassi = %s"
        values = [row[col] for col in colunas_comuns] + [row['chassi']]
        cursor.execute(update_query, values)
    
    engine.commit()
    print(f"Atualização concluída! {len(df_csv_comuns)} registros atualizados.")
except Exception as e:
    engine.rollback()
    print(f"Erro durante atualização: {e}")
    raise
finally:
    cursor.close()

Atualização concluída! 649 registros atualizados.


In [38]:
df_pdi.columns

Index(['origem', 'nome_empresa', 'chv_empresa', 'bandeira_bi',
       'ordem_bandeira', 'sistema', 'revenda', 'idnfexterno',
       'numero_nota_fiscal', 'serie_nota_fiscal', 'status', 'novo_usado',
       'dta_processamento', 'dta_cancelamento_nota', 'tipo_transacao',
       'des_tipo_transacao', 'dta_entrada_saida', 'cod_cliente',
       'nome_cliente', 'tipo', 'subtipo_transacao', 'contador', 'cgccpf',
       'veiculo', 'chassi', 'placa', 'marca', 'modelo', 'des_modelo',
       'cod_familia', 'familia', 'tot_mercadoria', 'tot_custo_medio',
       'val_base_ipi', 'tot_ipi', 'tot_nota_fiscal', 'val_base_icms',
       'val_icms', 'val_isento', 'val_outros_icms', 'valdesconto',
       'val_base_iss', 'val_frete', 'val_seguro', 'val_base_icm_retido',
       'dta_documento', 'pps', 'val_pdi', 'percentil_atacado',
       'percentil_tiktok', 'percentil_nps', 'percentil_visits',
       'percentil_snapon', 'percentil_mystery', 'percentil_chatbot',
       'percentil_download', 'dta_recebimento

In [6]:
M1 = 6.4
M2 = 6.2

M3 = (18 - max(M1, M2)) / 2
print(f"Nota necessária na M3: {M3}")

Nota necessária na M3: 5.8


In [10]:
import scipy.stats as stats

# Desempenho nas provas anteriores
nota_prova_m1 = 2.4  # de 6 pontos
nota_prova_m2 = 2.2  # de 6 pontos (já que M2 = 6.2 e trabalho = 4.0)

# Convertendo para escala de 0 a 10
nota_prova_m1_escala10 = (nota_prova_m1 / 6) * 10
nota_prova_m2_escala10 = (nota_prova_m2 / 6) * 10

# Média e desvio padrão do desempenho
media_provas = (nota_prova_m1_escala10 + nota_prova_m2_escala10) / 2
desvio_padrao = abs(nota_prova_m1_escala10 - nota_prova_m2_escala10) / 2

# Se o desvio for muito pequeno, usar um valor mínimo
if desvio_padrao < 0.5:
    desvio_padrao = 1.0

# Calculando a probabilidade de tirar >= 5.8
z_score = (5.8 - media_provas) / desvio_padrao
probabilidade = 1 - stats.norm.cdf(z_score)

print(f"Nota na prova M1 (escala 0-10): {nota_prova_m1_escala10:.2f}")
print(f"Nota na prova M2 (escala 0-10): {nota_prova_m2_escala10:.2f}")
print(f"Média das provas anteriores: {media_provas:.2f}")
print(f"Desvio padrão estimado: {desvio_padrao:.2f}")
print(f"\nNota necessária na M3: {M3:.2f}")
print(f"\nProbabilidade de tirar >= {M3:.2f} na P3: {probabilidade*100:.2f}%")

# Análise adicional
melhoria_necessaria = M3 - media_provas
print(f"\nMelhoria necessária em relação à média: {melhoria_necessaria:.2f} pontos")
print(f"Isso representa um aumento de {(melhoria_necessaria/media_provas)*100:.1f}%")

Nota na prova M1 (escala 0-10): 4.00
Nota na prova M2 (escala 0-10): 3.67
Média das provas anteriores: 3.83
Desvio padrão estimado: 1.00

Nota necessária na M3: 5.80

Probabilidade de tirar >= 5.80 na P3: 2.46%

Melhoria necessária em relação à média: 1.97 pontos
Isso representa um aumento de 51.3%


**Observação sobre a P3:**

Apesar da análise estatística indicar uma chance mínima de passar, é importante considerar que o grau de dificuldade da P3 pode ser diferente. Muitas vezes, a FAE ajusta a prova final para facilitar a aprovação dos alunos. Portanto, não desanime totalmente com os números — fatores externos e ajustes pedagógicos podem aumentar suas chances!

**Resumo final:**

Apesar de possíveis facilidades na P3, considerando o histórico e a estatística, a situação realmente está complicada. É importante se preparar ao máximo e, se possível, buscar alternativas para garantir a aprovação.